# TensorFlow Recommenders: Quickstart

In this tutorial, we build a simple matrix factorization model using the MovieLens 100K dataset with TFRS. We can use this model to recommend movies for a given user.

## Import TFRS
First, install and import TFRS:

In [2]:
!pip install --upgrade pip

In [3]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

In [4]:
from typing import Dict, Text

import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

# Read the data

https://grouplens.org/datasets/movielens/100k/

In [5]:
# Ratings data.
ratings = tfds.load('movielens/100k-ratings', split="train")
# Features of all the available movies.
movies = tfds.load('movielens/100k-movies', split="train")

# Select the basic features.
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"]
})
movies = movies.map(lambda x: x["movie_title"])

In [6]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

Build vocabularies to convert user ids and movie titles into integer indices for embedding layers:

In [7]:
user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))

movie_titles_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies)

## Define a model
We can define a TFRS model by inheriting from tfrs.Model and implementing the compute_loss method:

In [8]:
class MovieLensModel(tfrs.Model):
  # We derive from a custom base class to help reduce boilerplate. Under the hood,
  # these are still plain Keras Models.

  def __init__(
      self,
      user_model: tf.keras.Model,
      movie_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()

    # Set up user and movie representations.
    self.user_model = user_model
    self.movie_model = movie_model

    # Set up a retrieval task.
    self.task = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed.

    user_embeddings = self.user_model(features["user_id"])
    movie_embeddings = self.movie_model(features["movie_title"])

    return self.task(user_embeddings, movie_embeddings)

Define the two models and the retrieval task.

In [10]:
# Define user and movie models.
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(), 64)
])
movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocab_size(), 64)
])

# Define your objectives.
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    movies.batch(128).map(movie_model)
  )
)

## Fit and evaluate it.
Create the model, train it, and generate predictions:

In [11]:
# Create a retrieval model.
model = MovieLensModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# Train for 3 epochs.
model.fit(ratings.batch(4096), epochs=3)

# Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    movies.batch(100).map(lambda title: (title, model.movie_model(title))))

# Get some recommendations.
_, titles = index(np.array(["42"]))
print(f"Top 3 recommendations for user 42: {titles[0, :3]}")

Epoch 1/3
25/25 [==============================] - 4s 141ms/step - factorized_top_k/top_1_categorical_accuracy: 6.0000e-05 - factorized_top_k/top_5_categorical_accuracy: 0.0016 - factorized_top_k/top_10_categorical_accuracy: 0.0049 - factorized_top_k/top_50_categorical_accuracy: 0.0437 - factorized_top_k/top_100_categorical_accuracy: 0.0985 - loss: 33080.3480 - regularization_loss: 0.0000e+00 - total_loss: 33080.3480
Epoch 2/3
25/25 [==============================] - 4s 141ms/step - factorized_top_k/top_1_categorical_accuracy: 1.4000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0050 - factorized_top_k/top_10_categorical_accuracy: 0.0141 - factorized_top_k/top_50_categorical_accuracy: 0.1057 - factorized_top_k/top_100_categorical_accuracy: 0.2116 - loss: 31014.7544 - regularization_loss: 0.0000e+00 - total_loss: 31014.7544
Epoch 3/3
25/25 [==============================] - 3s 133ms/step - factorized_top_k/top_1_categorical_accuracy: 3.8000e-04 - factorized_top_k/top_5_categorica

In [12]:
cached_test = test.batch(4096).cache()

In [13]:
model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 2s 142ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0059 - factorized_top_k/top_5_categorical_accuracy: 0.0383 - factorized_top_k/top_10_categorical_accuracy: 0.0723 - factorized_top_k/top_50_categorical_accuracy: 0.2569 - factorized_top_k/top_100_categorical_accuracy: 0.3941 - loss: 29229.2311 - regularization_loss: 0.0000e+00 - total_loss: 29229.2311


{'factorized_top_k/top_1_categorical_accuracy': 0.005900000222027302,
 'factorized_top_k/top_5_categorical_accuracy': 0.03830000013113022,
 'factorized_top_k/top_10_categorical_accuracy': 0.07225000113248825,
 'factorized_top_k/top_50_categorical_accuracy': 0.256850004196167,
 'factorized_top_k/top_100_categorical_accuracy': 0.39410001039505005,
 'loss': 26507.771484375,
 'regularization_loss': 0,
 'total_loss': 26507.771484375}